In [1]:
import requests,re,pymysql,os,time
from bs4 import BeautifulSoup,Comment
from langconv import *
from fake_useragent import UserAgent

ua = UserAgent().random
header = {
        'Referer': 'http://www.guoxuedashi.com/',
        'User-Agent': ua,
        'Connection':'close'
}

# 得到二级目录
def secondfloor(url):
    page = requests.get(url, headers=header, verify=False)
    soup = BeautifulSoup(page.text).find('div',class_='info_cate clearfix').find_all('a')
    site = 'http://www.guoxuedashi.com'
    sf = []
    for i in soup:
        link = site+i['href']
        name = wash(str(i.string))
        sf.append([wash(name),link])
    return sf

# 得到正文
def gettext(url):
    page = requests.get(url, headers=header, verify=False, timeout=6.5)
    soup = BeautifulSoup(page.text).find('div',class_='info_txt clearfix')

    comments = soup.findAll(text=lambda text: isinstance(text, Comment))
    [comment.extract() for comment in comments]

    return wash(soup.text)

# 清洗
def is_chinese(uchar):
    if (uchar >= '\u4e00' and uchar <= '\u9fa5') or uchar in ('，','。','：','：' '\n'):
        return True
    else:
        return False

def reserve_chinese(content):
    content_str = ''
    for i in content:
        if is_chinese(i):
            content_str += i
    return content_str

def wash(text):
    t = re.sub(u'\\(.*?\\)|\\〔.*?\\〕|\\（.*?\\）|\\【.*?\\】|\\[.*?\\]|\\{.*?\\}|\\｛.*?\\｝','',text)
    return reserve_chinese(T2S(t))

def T2S(sentence):
    sentence = Converter('zh-hans').convert(sentence)
    return sentence

# log去重
def repeatRemove(target):
    text = []
    with open('E:/材料/四库全书/'+target,'r') as logfile:
        logs = logfile.readlines()
        for log in logs:
            if log not in text:
                text.append(log)
    with open('E:/材料/四库全书/'+target,'w') as logfile:
        for log in text:
            logfile.write(log)

In [ ]:
# 主程序
ff = []
with open('E:/材料/四库全书/目录.txt','r') as content:
    r = content.readlines()
    for i in r:
        ff.append(i.replace('\n','').split('   :::   '))
total = str(len(ff))
count = 0

try:
    filedir = 'E:/材料/四库全书'
    print(time.strftime('%Y-%m-%d %H:%M:%S')+'  ========================================= Start =========================================')
    for index in ff:
        count += 1
        countstr = '[ '+str(count)+' / '+total+' ] '
        name = index[0].replace('《','').replace('》','')
        link = index[1]
        try:
            dirname = 'E:/材料/四库全书/'+name
            if os.path.exists(dirname):
                continue
            os.mkdir(dirname)

            # 获取二级目录
            sf = secondfloor(link)
            with open(dirname+'/目录.txt','w') as content:
                for i in sf:
                    subname = i[0]
                    sublink = i[1]
                    print(time.strftime('%Y-%m-%d %H:%M:%S ')+countstr+name+' - '+subname)
                    try:
                        with open(dirname+'/'+subname+'.txt','w') as file:
                            final_text = gettext(sublink)
                            file.write(final_text)
                            # time.sleep(1)
                        content.write(subname+'   :::   '+sublink+'\n')
                    except Exception as e:
                        print(countstr+name+' - '+subname,e)
                        with open('E:/材料/四库全书/log.txt','a+') as log:
                            log.write(name+'   :::   '+link+'\n')   

            print(time.strftime('%Y-%m-%d %H:%M:%S  ')+'=============================='*3)

        except Exception as e:
            print(name,link,e)
            with open('E:/材料/四库全书/log.txt','a+') as log:
                log.write(name+'   :::   '+link+'\n')
        
    print(time.strftime('%Y-%m-%d %H:%M:%S')+'  ========================================= Finish =========================================')
            
except KeyboardInterrupt as e:
    print(name,link,'  ===================================== KeyboardInterrupt =====================================')
    with open('E:/材料/四库全书/log.txt','a+') as log:
        log.write(name+'   :::   '+link+'\n')            
except Exception as e:
    print(name,link,'  =========================================== Error ===========================================')
    with open('E:/材料/四库全书/log.txt','a+') as log:
        log.write(name+'   :::   '+link+'\n')

In [4]:
# 补漏
target = 'log.txt'
logger = 'log2.txt'

repeatRemove(target)
tasks = []
with open('E:/材料/四库全书/'+target,'r') as log:
    leaks = log.readlines()
    for i in leaks:
        task = i.replace('\n','').split('   :::   ')
        if task not in tasks:
            tasks.append(task)
for task in tasks:
    print('待补：'+task[0])
filedir = 'E:/材料/四库全书/'
print(time.strftime('%Y-%m-%d %H:%M:%S')+'  ========================================== Start ==========================================')

for task in tasks:
    name,link = task[0],task[1]
    seconddir = filedir + name +'/'
    try:
        sf = secondfloor(link)
        with open(seconddir+'目录.txt','a') as content:
            for i in sf:
                subname = i[0]
                sublink = i[1]
                if (not os.path.exists(seconddir+subname+'.txt')) or os.path.getsize(seconddir+subname+'.txt')==0:
                    print(time.strftime('%Y-%m-%d %H:%M:%S')+'  补漏：'+name+' - '+subname)
                    try:
                        with open(seconddir+subname+'.txt','w') as file:
                            final_text = gettext(sublink)
                            file.write(final_text)
                            # time.sleep(1)
                        content.write(subname+'   :::   '+sublink+'\n')
                    except Exception as e:
                        print(name+' - '+subname,e)
                        with open('E:/材料/四库全书/'+logger,'a+') as log:
                            log.write(name+'   :::   '+link+'\n')
        print(time.strftime('%Y-%m-%d %H:%M:%S  ')+'====================================='*3)

    except Exception as e:
        print(name,link,e)
        with open('E:/材料/四库全书/'+logger,'a+') as log:
            log.write(name+'   :::   '+link+'\n')
with open('E:/材料/四库全书/'+target,'w') as e:
    pass
print(time.strftime('%Y-%m-%d %H:%M:%S')+'  ========================================== Finish ==========================================')

2020-03-06 16:55:58  ========================================== Start ==========================================
2020-03-06 16:55:59  ========================================== Finish ==========================================


In [ ]:
# 得到一级目录.txt
url = 'http://www.guoxuedashi.com/SiKuQuanShu/'
page = requests.get(url)
soup = BeautifulSoup(page.text).find('div',class_='info_content zj clearfix').find_all('table',class_='table2')

site = 'http://www.guoxuedashi.com'
with open('E:/材料/四库全书/目录.txt','w') as content:
    for i in soup:
        item = i.find_all('a')
        for j in item:
            link = site+j['href']
            name = wash(str(j.string))
            #ff.append([name,link])
            content.write(name+'   :::   '+link+'\n')
